In [1]:
import pandas as pd
import datetime
from datetime import time,timedelta

In [2]:
record = pd.read_csv('raw_attendance.csv')

In [3]:
print(record)

     Unnamed: 0               Full Name    User Action               Timestamp
0             0  hXcMUMjoQg_2014GOI4381  Joined before  9/26/2020, 10:56:12 AM
1             1  hXcMUMjoQg_2014GOI4381           Left  9/26/2020, 10:57:10 AM
2             2  ZcwOpnilJX_4537ITK4802  Joined before  9/26/2020, 10:56:12 AM
3             3  ZcwOpnilJX_4537ITK4802           Left  9/26/2020, 10:59:03 AM
4             4  ZcwOpnilJX_4537ITK4802         Joined  9/26/2020, 10:59:04 AM
..          ...                     ...            ...                     ...
555         555  laCHGVZsdO_7088FTO2914         Joined  9/26/2020, 12:22:04 PM
556         556  laCHGVZsdO_7088FTO2914           Left  9/26/2020, 12:24:57 PM
557         557  QMTFYtPXpQ_9003JMW3329         Joined  9/26/2020, 12:22:20 PM
558         558  LvEaopzuHM_4872WDB5314         Joined  9/26/2020, 12:26:13 PM
559         559  LvEaopzuHM_4872WDB5314           Left  9/26/2020, 12:26:30 PM

[560 rows x 4 columns]


In [4]:
num = len(record.Timestamp)

In [5]:
print(num)

560


In [6]:
d={}
final={'Name':[],'Entrynumber':[],'Timestamp':[],'Totaltime':[],'Status':[]}

In [7]:
for i in range(num):
    curr=record.loc[i]
    name=curr[1]
    action=curr[2]
    timestamp=curr[3]
    date_time=timestamp
    name=name.split('_')
    timestamp=timestamp.split(',')
    #print(type(timestamp))
    if name[0] in d.keys():
        if action == "Joined":
            t = d[name[0]][1]+timestamp[1]+","
            d[name[0]][1]=t
        elif action == "Left":
            t=d[name[0]][1]
            s=t.split(",")
            initialtime=s[-2]
            finaltime=timestamp[1]
            initialtime=datetime.datetime.strptime(initialtime," %I:%M:%S %p")
            finaltime=datetime.datetime.strptime(finaltime," %I:%M:%S %p")
            difference=finaltime-initialtime
            duration=difference.total_seconds()/60
            d[name[0]][1]=t+timestamp[1]+","+str(round(duration,2))+'\n,'
            d[name[0]][2]=d[name[0]][2]+duration
            if d[name[0]][2]>40:
                d[name[0]][3]='Present'
            
    else:
        d[name[0]]=[]
        if len(name)==1:
            d[name[0]].append("")
        else:
            d[name[0]].append(name[1])
        d[name[0]].append(timestamp[1]+",")
        d[name[0]].append(0)
        d[name[0]].append('Absent')
        
for key in d.keys():
    s=d[key][1]
    if s[-1]==',' and s[-2]!='\n':
        finaltime=" 12:30:00 PM"
        initialtime= s.split(',')[-2]
        initialtime=datetime.datetime.strptime(initialtime," %I:%M:%S %p")
        _final=datetime.datetime.strptime(finaltime," %I:%M:%S %p")
        difference=_final-initialtime
        duration=difference.total_seconds()/60
        d[key][1]=d[key][1]+finaltime+","+str(round(duration,2))+'\n'
        d[key][2]=d[key][2]+duration
        if d[key][2]>40:
            d[key][3]='Present'

In [8]:
for key in d.keys():
    print(type(key))
    print(type(d[key][0]))
    print(type(d[key][1]))
    print(type(d[key][2]))
    print(type(d[key][3]))
    break
    

<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>


In [9]:
for key in d.keys():
    #print(key)
    final['Name'].append(key)
    final['Entrynumber'].append(d[key][0])
    final['Timestamp'].append(d[key][1])
    final['Totaltime'].append(str(round(d[key][2],2)))
    final['Status'].append(d[key][3])

In [10]:
df = pd.DataFrame(final)

In [11]:
print(df)

           Name  Entrynumber  \
0    hXcMUMjoQg  2014GOI4381   
1    ZcwOpnilJX  4537ITK4802   
2    DeKThdfHPS  3698HFP8237   
3    MHSkvimJDu  1783MCY5605   
4    CENUJlflMr  6174IFB4290   
..          ...          ...   
180  xzdOqziODg  1647CUT0284   
181  OdPQABIWmc  4285MGX8767   
182  NjQCIWxhIi  1098DOD3024   
183  VEMdKyOqDI  1596QYK5980   
184  laCHGVZsdO  7088FTO2914   

                                             Timestamp Totaltime   Status  
0     10:56:12 AM, 10:57:10 AM,0.97\n, 11:00:13 AM,...     89.18  Present  
1     10:56:12 AM, 10:59:03 AM,2.85\n, 10:59:04 AM,...    128.58  Present  
2     10:56:12 AM, 10:57:27 AM,1.25\n, 10:57:47 AM,...     16.77   Absent  
3     10:56:12 AM, 10:56:18 AM,0.1\n, 10:56:27 AM, ...     87.53  Present  
4     10:56:12 AM, 10:59:05 AM,2.88\n, 10:59:06 AM,...     88.83  Present  
..                                                 ...       ...      ...  
180                  11:56:03 AM, 12:06:08 PM,10.08\n,     10.08   Absent  
181   1

In [12]:
df.to_csv('attendance.csv')